In [8]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, gwTxP, ret_count, gwPll):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --gatewayTxPriority="+f"{gwTxP}"+" --endDeviceMaxRetransmissions="+f"{ret_count}"+" --gatewayReceptionPaths="+f"{gwPll}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [9]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [10]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [11]:
import concurrent.futures
import json
import math
import os

def run_scenario(traffic_type, realistic_model, device_class, scenario_name, gwTxP, ret_count, gwPll):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            # run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, gwTxP, ret_count, gwPll)
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, gwTxP, ret_count, gwPll)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_best_params'
    test_sub_folder = 'on'

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [12]:
import threading

threads = []


In [13]:

# run_scenario(traffic_type, realistic_model, device_class, scenario_name, gwTxP, ret_count, gwPll):

#Unconfirmed traffic for Class B and Class C devices
threads.append(threading.Thread(target=run_scenario, args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", True, 8, 8)))
threads.append(threading.Thread(target=run_scenario, args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", True, 8, 8)))

#Confirmed traffic for Class B and Class C devices
threads.append(threading.Thread(target=run_scenario, args=(1, False, 'B', "ULPDR_CONFIRMED_B_ULPDR", False, 8, 16)))
threads.append(threading.Thread(target=run_scenario, args=(1, False, 'B', "ULPDR_CONFIRMED_B_CPSR", True, 8, 16)))
threads.append(threading.Thread(target=run_scenario, args=(1, False, 'C', "ULPDR_CONFIRMED_C", True, 8, 16)))

#Mixed traffic for Class B and Class C devices
threads.append(threading.Thread(target=run_scenario, args=(2, False, 'B', "ULPDR_MIXED_B_ULPDR", False, 8, 16)))
threads.append(threading.Thread(target=run_scenario, args=(2, False, 'B', "ULPDR_MIXED_B_CPSR", True, 8, 16)))
threads.append(threading.Thread(target=run_scenario, args=(2, False, 'C', "ULPDR_MIXED_C_ULPDR", True, 8, 16)))
threads.append(threading.Thread(target=run_scenario, args=(2, False, 'C', "ULPDR_MIXED_C_CPSR", True, 8, 16)))

In [14]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status Best Params', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=8 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
ULPDR_UNCONFIRMED_B, Simualtion with device count: 1
ULPDR_CONFIRMED_B_ULPDR, Simualtion with device count: 1
ULPDR_CONFIRMED_B_CPSR, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=8 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
ULPDR_CONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-exampl

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=16 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=32 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 39
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=16 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=39 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 49
ULPDR_MIXED_B_CPSR, Simualtion with device count: 25
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=16 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=20 --dataUpType=2 --realisticModel=False"
/home/adi/Workspace/n


+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuratio

ULPDR_CONFIRMED_B_CPSR, Simualtion with device count: 49
ULPDR_MIXED_C_CPSR, Simualtion with device count: 39
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=16 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=32 --dataUpType=2 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=8 --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=49 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 60
ULPDR_UNCONFIRMED_B, Simualtion with device count: 60
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gatewayTxPriority=True --endDeviceMaxRetransmissions=8 --gatewayReceptionPaths=8 --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=49 -

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
0.9705882352941176 L: 0.085
1.0 L: 0.10375
1.0 L: 0.11875
1.0 L: 0.1475
1.0 L: 0.1875
1.0 L: 0.23
1.0 L: 0.29125
1.0 L: 0.3625
1.0 L: 0.42875
0.9929742388758782 L: 0.53375
0.9864077669902913 L: 0.64375
0.9861751152073732 L: 0.81375
0.9659949622166247 L: 0.9925
0.9704081632653061 L: 1.225
0.9546979865771812 L: 1.49
0.9239920687376074 L: 1.89125
0.8957776590058792 L: 2.33875
0.8698659749243407 L: 2.89125
0.8200138985406532 L: 3.5975
0.7468531468531469 L: 4.46875
0.6687214611872146 L: 5.475
0.5938704349421913 L: 6.81125


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
1.0 L: 0.2475
1.0 L: 0.30375
1.0 L: 0.3875
1.0 L: 0.45375
1.0 L: 0.5625
0.9946902654867257 L: 0.70625
0.9943019943019943 L: 0.8775
0.977725674091442 L: 1.06625
0.9896616541353384 L: 1.33
0.9711465451784358 L: 1.64625
0.9508196721311475 L: 2.05875
0.9287445467765391 L: 2.57875
0.8900235663786331 L: 3.1825
0.833648393194707 L: 3.9675
0.7588580168238593 L: 4.90375
0.6720530350113942 L: 6.03375
0.5803497085761865 L: 7.50625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
0.9949748743718593 L: 0.24875
1.0 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9844444444444445 L: 0.5625
0.9787610619469026 L: 0.70625
0.9857549857549858 L: 0.8775
0.9742086752637749 L: 1.06625
0.9755639097744361 L: 1.33
0.9582384206529992 L: 1.64625
0.9216757741347905 L: 2.05875
0.9001454192922927 L: 2.57875
0.8534956794972506 L: 3.1825
0.800252047889099 L: 3.9675
0.7280142747897017 L: 4.90375
0.6461570333540502 L: 6.03375
0.5650291423813488 L: 7.50625
3294.9259543418884
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
